# AI for stock market prediction: Using LLMs for TimeSeries Predictions (LLMTime implementation)

Project by: Jana Nikolovska <br>
Supervised by: Giacomo Frisoni, MSc <br><!--  -->
Prof. Gianluca Moro, PhD <br>

ALMA MATER STUDITORIUM - University of Bologna <br>
November 2025

---

**Summary:** <br>
In this project, I explore the use of Large Language Models (LLMs) for time series forecasting, focusing on the task of stock market prediction. The work was proposed and mentored by Prof. Gianluca Moro and Giacomo Frisoni at the University of Bologna.

As a starting point, I used a provided notebook by my mentors. The notebook introduces the dataset (**historical S&P 500 data via [`yfinance`](https://pypi.org/project/yfinance/)**), a basic linear regression model, which I chose not to include in the final results, that served as an early benchmark and highlighted the underlying complexity of the forecasting problem and finally, the **"Trading Protocol"** — a framework designed to evaluate forecasting performance through simulated trading strategies —that I extended upon to incorporate multiple strategy types and Monte Carlo-based robustness testing.

On the chosen dataset, a traditional **statistical model ARIMA** and **LLM-based forecasting approaches** have been tested and evaluated. For the implementation, I followed the methodology described in [this paper](https://arxiv.org/pdf/2310.07820) and its [official implementation](https://github.com/ngruver/llmtime/tree/main). The code from the paper has been adapted and extended with additional functionality tailored to the specific requirements of my experiments.

The problem was defined comparable to that of the referenced paper, particularly in terms of training and testing proportions. The task was defined such that, given 150 days of historical values, the objective was to predict the subsequent 29 days values following an autoregressive approach without direct access to ground truth during prediction.

Unlike the provided baseline notebook, which operated directly on price levels, this project instead utilized log returns as the primary time-series input. This choice was motivated by findings from the exploratory data analysis, which indicated that the raw price series exhibited non-stationary behavior — making a dificult target for predicting. Transforming the data into returns helped achieve approximate stationarity, giving a chance to  the models to capture relative changes more effectively.

As mentioned earlier, both statistical and trading-based evaluation methods were used, with greater emphasis placed on the trading metrics (Trading Protocol), while the statistical ones were included primarily for completeness and exploratory insight.

In [24]:
import os
os.environ['OMP_NUM_THREADS'] = '4'
import pandas as pd
import matplotlib.pyplot as plt
import openai
from data.serialize import SerializerSettings
from models.darts import get_arima_predictions_data
from models.llmtime import get_llmtime_predictions_data
import numpy as np
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

from src.utils import split_time_series
from src.evaluation import *
from src.visualizations import plot_predictions_vs_actual, plot_cumulative_returns_simulation, create_performance_summary_table
from src.models.llmtime import get_autotuned_predictions_data

# warnings.simplefilter('once', UserWarning)
# warnings.filterwarnings("ignore", category=ConvergenceWarning)

warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
with open("secrets/openai_key.txt", "r") as file:
    openai_api_key = file.read().strip()
openai.api_key = openai_api_key

In [3]:
PREDS = {}
RESULTS = []
LAG = 12
TRAIN_SIZE = 150
PREDICTION_SIZE = 29
RETURNS_PATH = os.path.join("data","sp500_returns_modeling.csv")
DS_NAME = 'SP500_returns'

## Model Setup and Implementation

In this section, the models are introduced, and the key hyperparameters to be tuned and validated are defined.

#### 📈 **ARIMA**
- Linear, statistical model  
- Assumes stationarity  
- Captures short-term autocorrelation  
- Limited with nonlinear or regime changes  

#### 🤖 **ChatGPT (LLM-based)**
- Transformer, deep learning approach  
- Should perform better with nonlinear and complex dependencies  
- No explicit statistical assumptions  
- Less interpretable

In [4]:
"""
    Function adapted from: https://github.com/ngruver/llmtime
    Original author(s): Nicholas Gruver et al.
"""
gpt4_hypers = dict(
    model="gpt-4",
    alpha=0.3,
    basic=True,
    temp=1.0,
    top_p=0.8,
    settings=SerializerSettings(base=10, prec=3, signed=True, time_sep=', ', bit_sep='', minus_sign='-')
)

gpt3_hypers = dict(
    model="gpt-3.5-turbo-instruct",
    temp=0.7,
    alpha=0.95,
    beta=0.3,
    basic=False,
    settings=SerializerSettings(base=10, prec=3, signed=True, half_bin_correction=True)
)
arima_hypers = dict(p=[12,30], d=[1,2], q=[0])

"""
    Hyperparameters derived from Exploatory analysis of data
"""
# GPT-4 with volatility-aware settings (higher precision for extreme values)
gpt4_volatility_aware = dict(
    model="gpt-4",
    alpha=0.1,  # More aggressive scaling for volatility clustering detection
    basic=False,  
    temp=0.3,  # Lower temperature for more focused predictions during volatile periods
    top_p=0.9,
    settings=SerializerSettings(base=10, prec=4, signed=True, time_sep=', ', bit_sep='', minus_sign='-')
)

# GPT-3.5 with regime-change sensitivity
gpt3_regime_sensitive = dict(
    model="gpt-3.5-turbo-instruct",
    temp=1.2,  # Higher creativity to capture regime changes
    alpha=0.8,  # Medium scaling for robustness
    beta=0.5,  # Higher beta for shift handling
    basic=False,
    settings=SerializerSettings(base=10, prec=3, signed=True, half_bin_correction=True, time_sep=', ')
)

# Conservative LLM configuration for stable periods
gpt4_conservative = dict(
    model="gpt-4",
    alpha=0.7,  # Conservative scaling
    basic=True,
    temp=0.1,  # Very low temperature for consistency
    top_p=0.7,
    settings=SerializerSettings(base=10, prec=2, signed=True, time_sep=', ', bit_sep='', minus_sign='-')
)

arima_short_term = dict(p=[1,2,3], d=[0,1], q=[0,1])  # For short-term dependencies
arima_medium_term = dict(p=[5,7,12], d=[1], q=[1,2])  # Medium-term patterns
arima_high_volatility = dict(p=[1,3,5], d=[1,2], q=[1,3])  # For volatile periods

In [5]:
# ======================================
# ARIMA Forecast
# ======================================
def arima_forecast(x, x_forecast, forecast_horizon: int = 5, hypers: dict = None) -> np.ndarray:
    """Forecast returns using ARIMA model"""
    h = forecast_horizon

    if hypers is None:
       raise ValueError("Missing hypers")
   
    preds = get_autotuned_predictions_data(
        train=x,
        test=x_forecast,
        hypers=hypers,
        num_samples=1,
        get_predictions_fn=get_arima_predictions_data,
        verbose=False,
        parallel=False
    )

    fc = np.array(preds.get("median", [0] * h), dtype=float)
    return fc


# ======================================
# LLM Forecast
# ======================================
def llm_forecast(x, x_forecast, forecast_horizon: int = 5, hypers: dict = None) -> np.ndarray:
    """Forecast returns using ARIMA model"""
    h = forecast_horizon

    if hypers is None:
       raise ValueError("Missing hypers")
   
    preds = get_autotuned_predictions_data(
        train=x,
        test=x_forecast,
        hypers=hypers,
        num_samples=1,
        get_predictions_fn=get_llmtime_predictions_data,
        verbose=False,
        parallel=False
    )

    fc = np.array(preds.get("median", [0] * h), dtype=float)
    return fc


## Evaluation

#### 💹 Trading Performance Evaluation

This function evaluates trading performance by simulating investment decisions based on return predictions. It is an extention of the "Trading Protocol" introduced in the University notebook.  
It supports multiple trading strategies:

- **Threshold Strategy:** Buy when the predicted return exceeds a specified threshold  
- **Relative Strategy:** Buy when the predicted return is above the median prediction  
- **Top Quartile Strategy:** Buy only when the predicted return is in the top 25% of all predictions  

The function calculates key metrics, including:

- **💰 Total Return:** Overall percentage gain or loss across the forecast period  
- **🎯 Directional Accuracy:** Proportion of times the model correctly predicted the direction of the return (up or down)  
- **📊 Percentage of Profitable Trades:** Share of executed trades that resulted in a positive return  


In [6]:
import numpy as np

def returns_trading_performance_with_strategy(
    predicted_returns,
    actual_returns,
    initial_capital=1000,
    strategy="threshold",
    threshold=0.001,
    verbose=True
):
    """
    Trading performance evaluation with multiple strategies.
    Supports either a single series (1D) or multiple series (list-of-lists).

    Args:
        predicted_returns: 1D array/list OR list of 1D arrays/lists
        actual_returns:    same shape structure as predicted_returns
        initial_capital:   starting capital per series (if multiple series, applied per series)
        strategy:          "threshold" | "relative" | "top_quartile"
        threshold:         used only for "threshold" strategy
        verbose:           print summary

    Returns:
        final_capital_agg, total_return_pct_agg, accuracy_agg, profitable_trades_pct_agg, trade_info
          - Aggregates are averages across series if multiple series are provided.
          - trade_info includes averages and a per-series breakdown.
    """

    def _run_single_series(pred, act):
        pred = np.array(pred)
        act  = np.array(act)

        # signals by strategy (computed per series)
        if strategy == "threshold":
            buy_signals = pred > threshold
        elif strategy == "relative":
            median_pred = np.median(pred)
            buy_signals = pred > median_pred
        elif strategy == "top_quartile":
            quartile_75 = np.percentile(pred, 75)
            buy_signals = pred >= quartile_75
        else:
            raise ValueError("Strategy must be 'threshold', 'relative', or 'top_quartile'")

        capital = initial_capital
        trades = []

        for i, (pred_ret, actual_ret) in enumerate(zip(pred, act)):
            if buy_signals[i]:
                capital += capital * actual_ret
                trades.append(actual_ret)
            else:
                trades.append(0.0)

        final_capital = float(capital)
        total_return_pct = (final_capital - initial_capital) / initial_capital

        # direction accuracy
        accuracy = float(np.mean(np.sign(pred) == np.sign(act)))

        # trades metrics
        actual_trades = np.array(trades)[buy_signals]
        if actual_trades.size > 0:
            profitable_trades_pct = float(np.mean(actual_trades > 0))
            avg_trade_return = float(np.mean(actual_trades))
            num_trades = int(buy_signals.sum())
        else:
            profitable_trades_pct = 0.0
            avg_trade_return = 0.0
            num_trades = 0

        return {
            "final_capital": final_capital,
            "total_return_pct": float(total_return_pct),
            "accuracy": float(accuracy),
            "profitable_trades_pct": float(profitable_trades_pct),
            "num_trades": num_trades,
            "avg_trade_return": float(avg_trade_return),
        }

    # Detect single-series vs multi-series
    is_nested = isinstance(predicted_returns, (list, tuple)) and len(predicted_returns) > 0 and isinstance(predicted_returns[0], (list, tuple, np.ndarray))

    if not is_nested:
        # Single series path (keep original behavior)
        res = _run_single_series(predicted_returns, actual_returns)

        if verbose:
            print(f"=== Returns Trading Performance ===")
            print(f"Strategy: {strategy.upper()}")
            if strategy == "threshold":
                print(f"Threshold: {threshold:.4f} ({threshold*100:.2f}%)")
            print(f"Initial Capital: ${initial_capital:.2f}")
            print(f"Final Capital: ${res['final_capital']:.2f}")
            print(f"Total Return: {res['total_return_pct']:.2%}")
            print(f"Direction Accuracy: {res['accuracy']:.2%}")
            print(f"Number of Trades: {res['num_trades']}")
            print(f"Profitable Trades: {res['profitable_trades_pct']:.2%}")
            if res['num_trades'] > 0:
                print(f"Average Trade Return: {res['avg_trade_return']:.4f} ({res['avg_trade_return']*100:.2f}%)")

        trade_info = {
            "num_trades": res["num_trades"],
            "avg_trade_return": res["avg_trade_return"],
            "strategy_used": strategy,
            "threshold_used": threshold if strategy == "threshold" else None
        }

        return (
            res["final_capital"],
            res["total_return_pct"],
            res["accuracy"],
            res["profitable_trades_pct"],
            trade_info
        )

    # Multi-series path (list of lists)
    if len(predicted_returns) != len(actual_returns):
        raise ValueError("predicted_returns and actual_returns must have the same number of series")

    per_series_results = []
    for pred_series, act_series in zip(predicted_returns, actual_returns):
        if len(pred_series) != len(act_series):
            raise ValueError("Each predicted/actual series pair must have the same length")
        per_series_results.append(_run_single_series(pred_series, act_series))

    # Aggregate by averaging across series
    final_capital_agg = float(np.mean([r["final_capital"] for r in per_series_results]))
    total_return_pct_agg = float(np.mean([r["total_return_pct"] for r in per_series_results]))
    accuracy_agg = float(np.mean([r["accuracy"] for r in per_series_results]))
    profitable_trades_pct_agg = float(np.mean([r["profitable_trades_pct"] for r in per_series_results]))
    avg_num_trades = float(np.mean([r["num_trades"] for r in per_series_results]))
    avg_trade_return = float(np.mean([r["avg_trade_return"] for r in per_series_results]))

    if verbose:
        print(f"=== Returns Trading Performance (Multi-Series) ===")
        print(f"Strategy: {strategy.upper()}  |  Series: {len(per_series_results)}")
        if strategy == "threshold":
            print(f"Threshold: {threshold:.4f} ({threshold*100:.2f}%)")
        print(f"Initial Capital (per series): ${initial_capital:.2f}")
        print(f"Avg Final Capital: ${final_capital_agg:.2f}")
        print(f"Avg Total Return: {total_return_pct_agg:.2%}")
        print(f"Avg Direction Accuracy: {accuracy_agg:.2%}")
        print(f"Avg #Trades: {avg_num_trades:.2f}")
        print(f"Avg Profitable Trades: {profitable_trades_pct_agg:.2%}")
        if avg_num_trades > 0:
            print(f"Avg Trade Return: {avg_trade_return:.4f} ({avg_trade_return*100:.2f}%)")

    trade_info = {
        "num_trades": avg_num_trades,                # averaged for compatibility
        "avg_trade_return": avg_trade_return,        # averaged for compatibility
        "strategy_used": strategy,
        "threshold_used": threshold if strategy == "threshold" else None,
        "per_series": per_series_results             # detailed breakdown if you need it
    }

    return (
        final_capital_agg,
        total_return_pct_agg,
        accuracy_agg,
        profitable_trades_pct_agg,
        trade_info
    )


#### ⚙️ Prediction and Monte Carlo Trading Evaluation

This function calculates **statistical and trading performance metrics** for each forecasting model.

**Statistical Metrics**: <br>
- **📉 RMSE (Root Mean Squared Error):** Measures average magnitude of prediction errors  
- **📊 MAE (Mean Absolute Error):** Captures the average absolute difference between predicted and actual returns  
- **📈 R² (Coefficient of Determination):** Indicates how much of the variance in actual returns is explained by the model

**Trading Protocol metrics**: <br>
Then, for each of the trading strategies mentioned in the above function —  
runs **1,000 Monte Carlo simulations**, adding small Gaussian noise to the predictions to model uncertainty and averages the trading metrics, providing a robust estimate of model performance.


In [21]:
def predict_returns_with_trading(
    train_data,
    model_type: str = "arima",
    forecast_horizon: int = 30,
    hypers: dict = None,
    initial_capital: float = 10_000,
    strategies = ("threshold", "relative", "top_quartile"),
    monte_carlo_runs: int = 1000,
    noise_std: float = 0.001,      # 0.1% prediction noise per run
    seed: int = 42,                # reproducibility
    verbose: bool = True
):
    """
    Prediction + Monte Carlo trading evaluation BY STRATEGY.
    Runs `monte_carlo_runs` simulations **for each strategy**, then reports/returns averages per strategy.

    Args:
        splits
        model_type: "arima" or "llm"
        forecast_horizon: number of periods to forecast
        hypers: hyperparameters for the forecasting model
        initial_capital: starting capital for trading simulation
        strategies: iterable of strategy names to test (each run independently)
        monte_carlo_runs: number of runs per strategy
        noise_std: std-dev of Gaussian noise added to predictions each run
        seed: RNG seed for reproducibility
        verbose: print detailed output
    Returns:
        dict with:
            - 'model': str
            - 'statistical_metrics': dict
            - 'by_strategy': {
                strategy_name: {
                    'summary': dict of averaged metrics,
                    'runs': pd.DataFrame of all runs (per-run metrics)
                }, ...
              }
            - 'predictions': np.ndarray (base, noiseless)
            - 'actual': np.ndarray (true future)
    """
    if verbose:
        print(f" Model selected: {model_type.upper()}")
        print(f" Using real S&P 500 returns data")
        print(f" Monte Carlo: {monte_carlo_runs} runs per strategy")
        print(f" Noise on predictions: N(0, {noise_std})")

    # Train-test split
    data_history = list(s[0].values for s in train_data) 
    data_true_forecast = list(s[1].values for s in train_data)

    # --- Make base predictions (noiseless baseline used as center) ---
    if model_type.lower() == "arima":
        preds_base = arima_forecast(data_history,data_true_forecast, forecast_horizon=forecast_horizon, hypers=hypers)
    elif model_type.lower() == "llm":
        preds_base = llm_forecast(data_history, data_true_forecast, forecast_horizon=forecast_horizon, hypers=hypers)
    else:
        raise ValueError("model_type must be 'arima' or 'llm'")

    # --- Standard metrics ONCE (noiseless) ---
    true_vals = data_true_forecast

    metrics = calculate_metrics(true_vals, preds_base)

    print(f"\n ===== {model_type.upper()} Statistical Metrics (noiseless) =====")
    print(f"RMSE: {metrics['RMSE']:.6f}")
    print(f"MAE: {metrics['MAE']:.6f}")
    print(f"R²: {metrics['R2']:.4f}")

    # --- Monte Carlo BY STRATEGY ---
    rng = np.random.default_rng(seed)
    by_strategy = {}

    for strategy in strategies:
        print(f"\n ===== Strategy: {strategy} — {monte_carlo_runs} runs =====")

        run_records = []
        for _ in range(monte_carlo_runs):
            # Add Gaussian noise to base predictions to simulate forecast uncertainty
            noisy_preds = [
                np.array(p) + rng.normal(0.0, noise_std, size=len(p))
                for p in preds_base
            ]


            final_capital, total_return, accuracy, profitable_pct, trade_info = \
                returns_trading_performance_with_strategy(
                    predicted_returns=noisy_preds,
                    actual_returns=true_vals,
                    initial_capital=initial_capital,
                    strategy=strategy,
                    threshold=0.0005,   # 0.05%
                    verbose=False
                )

            run_records.append({
                "final_capital": final_capital,
                "total_return": total_return,
                "directional_acc": accuracy,
                "profitable_pct": profitable_pct,
                "num_trades": trade_info.get("num_trades", np.nan),
                "avg_trade_return": trade_info.get("avg_trade_return", np.nan),
            })

        df_runs = pd.DataFrame(run_records)

        # Averages & basic dispersion
        summary = {
            "avg_final_capital": float(df_runs["final_capital"].mean()),
            "std_final_capital": float(df_runs["final_capital"].std(ddof=1)),
            "avg_total_return": float(df_runs["total_return"].mean()),
            "std_total_return": float(df_runs["total_return"].std(ddof=1)),
            "avg_directional_acc": float(df_runs["directional_acc"].mean()),
            "avg_profitable_pct": float(df_runs["profitable_pct"].mean()),
            "avg_num_trades": float(df_runs["num_trades"].mean()),
            "avg_trade_return": float(df_runs["avg_trade_return"].mean()),
            # a few risk percentiles
            "p05_total_return": float(df_runs["total_return"].quantile(0.05)),
            "p50_total_return": float(df_runs["total_return"].quantile(0.50)),
            "p95_total_return": float(df_runs["total_return"].quantile(0.95)),
        }

        
        print(f" Average Total Return: {summary['avg_total_return']:.2%} "
                  f"(± {summary['std_total_return']:.2%}, p05 {summary['p05_total_return']:.2%}, "
                  f"median {summary['p50_total_return']:.2%}, p95 {summary['p95_total_return']:.2%})")
        print(f" Average Directional Accuracy: {summary['avg_directional_acc']:.2%}")
        print(f" Average Profitable Trades: {summary['avg_profitable_pct']:.2%}")
        print(f" Average Final Capital: ${summary['avg_final_capital']:.2f} "
                  f"(± ${summary['std_final_capital']:.2f})")
        print(f" Average #Trades: {summary['avg_num_trades']:.2f} | "
                  f"Avg Trade Return: {summary['avg_trade_return']:.4%}")

        by_strategy[strategy] = {
            "summary": summary,
            "runs": df_runs
        }

    results = {
        "model": model_type.upper(),
        "statistical_metrics": metrics,
        "by_strategy": by_strategy,
        "predictions": preds_base,
        "actual": true_vals
    }

    return results

## Running the Models and Analyzing Results

In this section, we test and compare the three forecasting models. Each model is explored across different **subgroups of hyperparameter configurations** and chosen the **optimal hyperparameters** for each of them. Once the best configurations are determined, we apply the **evaluation framework** previously introduced to measure both  statistical accuracy and trading performance under the defined strategies.

#### Loading the dataset

In [8]:
import random 

# Load returns data (stationary dataset from EDA)
print("⚙️ Loading returns dataset...")
returns_data = pd.read_csv(RETURNS_PATH)

print(f"Dataset shape: {returns_data.shape}")
print(f"Date range: {returns_data.index.min()} to {returns_data.index.max()}")
print("\nFirst few returns:")
print(returns_data.head())

# Create splits for returns forecasting
returns_series = returns_data['returns']
splits = split_time_series(returns_series, TRAIN_SIZE, PREDICTION_SIZE)

print("=" * 60)

print(f" 🧩Created {len(splits)} train-test splits for returns forecasting")
print(f"Each split - Train size: {TRAIN_SIZE}, Test size: {PREDICTION_SIZE}")
print("\n Sample split structure:")
print("Train data (last 5 values):")
print(splits[0][0].tail())
print(type(splits[0][0]))
print("\nTest data (first 5 values):")
print(splits[0][1].head())

⚙️ Loading returns dataset...
Dataset shape: (2890, 8)
Date range: 0 to 2889

First few returns:
                        date   returns        price        volume  \
0  2001-02-01 00:00:00-05:00  0.005461  1373.469971  1.118800e+09   
1  2001-02-02 00:00:00-05:00 -0.017474  1349.469971  1.048400e+09   
2  2001-02-03 00:00:00-05:00  0.001196  1351.083333  1.036600e+09   
3  2001-02-04 00:00:00-05:00  0.001194  1352.696696  1.024800e+09   
4  2001-02-05 00:00:00-05:00  0.001193  1354.310059  1.013000e+09   

   rolling_volatility  returns_lag1  returns_lag2  returns_lag3  
0            0.185403     -0.005620      0.007008      0.002258  
1            0.128971      0.005461     -0.005620      0.007008  
2            0.124988     -0.017474      0.005461     -0.005620  
3            0.095957      0.001196     -0.017474      0.005461  
4            0.095767      0.001194      0.001196     -0.017474  
 🧩Created 94 train-test splits for returns forecasting
Each split - Train size: 150, Test si

#### Running the Models and Obtaining Results

In [9]:
FORECAST_HORIZON = PREDICTION_SIZE
INITIAL_CAPITAL = 10000  # $10,000 starting capital

print(f" Initial Capital: ${INITIAL_CAPITAL:,}")
print(f" Forecast Horizon: {FORECAST_HORIZON} days")

 Initial Capital: $10,000
 Forecast Horizon: 29 days


In [10]:
# ======================================
# Test ARIMA Model
# ======================================

print("\n TESTING ARIMA MODEL")
print("=" * 50)
arima_models = {'ARIMA': arima_hypers,
                'ARIMA Short-Term': arima_short_term,
                'ARIMA Medium-Term': arima_medium_term,
                'ARIMA High-Volatility': arima_high_volatility}
all_arima_results = dict()
for model_name, params in arima_models.items():
    print(f"🧮 Model: {model_name}")
    all_arima_results[model_name] = predict_returns_with_trading(
        splits, 
        model_type="arima", 
        forecast_horizon=FORECAST_HORIZON,
        initial_capital=INITIAL_CAPITAL,
        hypers = params,
        verbose = False
    )
    print("=" * 40)


 TESTING ARIMA MODEL
🧮 Model: ARIMA
Starting hyperparameter tuning over 4 configurations...


Hyperparameter search:   0%|          | 0/4 [00:00<?, ?it/s]

Sampling with best hyper... defaultdict(<class 'dict'>, {'p': 12, 'd': 1, 'q': 0}) 
 with NLL -3.251371

 ===== ARIMA Statistical Metrics (noiseless) =====
RMSE: 0.008833
MAE: 0.006583
R²: -0.1744

 ===== Strategy: threshold — 1000 runs =====
 Average Total Return: -0.39% (± 0.14%, p05 -0.62%, median -0.39%, p95 -0.16%)
 Average Directional Accuracy: 49.03%
 Average Profitable Trades: 46.05%
 Average Final Capital: $9960.77 (± $13.97)
 Average #Trades: 13.45 | Avg Trade Return: -0.0399%

 ===== Strategy: relative — 1000 runs =====
 Average Total Return: -0.53% (± 0.20%, p05 -0.87%, median -0.53%, p95 -0.21%)
 Average Directional Accuracy: 49.01%
 Average Profitable Trades: 50.27%
 Average Final Capital: $9947.15 (± $20.06)
 Average #Trades: 14.00 | Avg Trade Return: -0.0373%

 ===== Strategy: top_quartile — 1000 runs =====
 Average Total Return: -0.52% (± 0.18%, p05 -0.82%, median -0.52%, p95 -0.23%)
 Average Directional Accuracy: 49.02%
 Average Profitable Trades: 49.25%
 Average Fina

Hyperparameter search:   0%|          | 0/12 [00:00<?, ?it/s]

Sampling with best hyper... defaultdict(<class 'dict'>, {'p': 1, 'd': 1, 'q': 1}) 
 with NLL -3.390996

 ===== ARIMA Statistical Metrics (noiseless) =====
RMSE: 0.008332
MAE: 0.006028
R²: -0.0404

 ===== Strategy: threshold — 1000 runs =====
 Average Total Return: -0.01% (± 0.19%, p05 -0.32%, median -0.01%, p95 0.30%)
 Average Directional Accuracy: 50.89%
 Average Profitable Trades: 49.04%
 Average Final Capital: $9999.20 (± $18.62)
 Average #Trades: 10.05 | Avg Trade Return: -0.0083%

 ===== Strategy: relative — 1000 runs =====
 Average Total Return: -0.07% (± 0.27%, p05 -0.50%, median -0.07%, p95 0.37%)
 Average Directional Accuracy: 50.83%
 Average Profitable Trades: 52.01%
 Average Final Capital: $9993.03 (± $26.66)
 Average #Trades: 14.00 | Avg Trade Return: -0.0046%

 ===== Strategy: top_quartile — 1000 runs =====
 Average Total Return: -0.03% (± 0.26%, p05 -0.45%, median -0.03%, p95 0.39%)
 Average Directional Accuracy: 50.85%
 Average Profitable Trades: 52.11%
 Average Final Ca

Hyperparameter search:   0%|          | 0/6 [00:00<?, ?it/s]

Sampling with best hyper... defaultdict(<class 'dict'>, {'p': 5, 'd': 1, 'q': 1}) 
 with NLL -3.363400

 ===== ARIMA Statistical Metrics (noiseless) =====
RMSE: 0.008344
MAE: 0.006047
R²: -0.0427

 ===== Strategy: threshold — 1000 runs =====
 Average Total Return: 0.04% (± 0.19%, p05 -0.26%, median 0.04%, p95 0.36%)
 Average Directional Accuracy: 50.73%
 Average Profitable Trades: 50.73%
 Average Final Capital: $10004.09 (± $18.84)
 Average #Trades: 10.28 | Avg Trade Return: -0.0073%

 ===== Strategy: relative — 1000 runs =====
 Average Total Return: 0.03% (± 0.26%, p05 -0.40%, median 0.03%, p95 0.44%)
 Average Directional Accuracy: 50.66%
 Average Profitable Trades: 51.85%
 Average Final Capital: $10002.57 (± $25.54)
 Average #Trades: 14.00 | Avg Trade Return: 0.0022%

 ===== Strategy: top_quartile — 1000 runs =====
 Average Total Return: 0.05% (± 0.23%, p05 -0.32%, median 0.06%, p95 0.42%)
 Average Directional Accuracy: 50.67%
 Average Profitable Trades: 51.85%
 Average Final Capital

Hyperparameter search:   0%|          | 0/12 [00:00<?, ?it/s]

Sampling with best hyper... defaultdict(<class 'dict'>, {'p': 1, 'd': 1, 'q': 1}) 
 with NLL -3.390996

 ===== ARIMA Statistical Metrics (noiseless) =====
RMSE: 0.008332
MAE: 0.006028
R²: -0.0404

 ===== Strategy: threshold — 1000 runs =====
 Average Total Return: -0.01% (± 0.19%, p05 -0.32%, median -0.01%, p95 0.30%)
 Average Directional Accuracy: 50.89%
 Average Profitable Trades: 49.04%
 Average Final Capital: $9999.20 (± $18.62)
 Average #Trades: 10.05 | Avg Trade Return: -0.0083%

 ===== Strategy: relative — 1000 runs =====
 Average Total Return: -0.07% (± 0.27%, p05 -0.50%, median -0.07%, p95 0.37%)
 Average Directional Accuracy: 50.83%
 Average Profitable Trades: 52.01%
 Average Final Capital: $9993.03 (± $26.66)
 Average #Trades: 14.00 | Avg Trade Return: -0.0046%

 ===== Strategy: top_quartile — 1000 runs =====
 Average Total Return: -0.03% (± 0.26%, p05 -0.45%, median -0.03%, p95 0.39%)
 Average Directional Accuracy: 50.85%
 Average Profitable Trades: 52.11%
 Average Final Ca

In [11]:
# ======================================
# Test GPT-3.5 Model
# ======================================

print("\n TESTING GPT-3.5 MODEL")
print("=" * 50)
gpt3_models = {'LLMTime GPT-3.5': gpt3_hypers, 'GPT-3.5 Regime-Sensitive': gpt3_regime_sensitive}
all_gpt3_results = dict()

for model_name, params in gpt3_models.items():
    print(f"🧮 Model: {model_name}")
    all_gpt3_results[model_name] = predict_returns_with_trading(
        splits, 
        model_type="llm", 
        forecast_horizon=FORECAST_HORIZON, 
        hypers=params,
        initial_capital=INITIAL_CAPITAL,
        verbose=False)
    print("=" * 40)


 TESTING GPT-3.5 MODEL
🧮 Model: LLMTime GPT-3.5
Starting hyperparameter tuning over 1 configurations...
Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gpt-3.5-turbo-instruct', 'temp': 0.7, 'alpha': 0.95, 'beta': 0.3, 'basic': False, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=' ,', bit_sep=' ', plus_sign='', minus_sign=' -', half_bin_correction=True, decimal_point='', missing_str=' Nan')}) 
 with NLL inf


100%|███████████████████████████████████████████████████████████████████████████| 94/94 [03:41<00:00,  2.36s/it]



 ===== LLM Statistical Metrics (noiseless) =====
RMSE: 0.013524
MAE: 0.010103
R²: -3.2078

 ===== Strategy: threshold — 1000 runs =====
 Average Total Return: -0.12% (± 0.08%, p05 -0.25%, median -0.12%, p95 0.01%)
 Average Directional Accuracy: 51.63%
 Average Profitable Trades: 54.22%
 Average Final Capital: $9987.89 (± $8.11)
 Average #Trades: 15.19 | Avg Trade Return: 0.0151%

 ===== Strategy: relative — 1000 runs =====
 Average Total Return: 0.08% (± 0.10%, p05 -0.09%, median 0.08%, p95 0.25%)
 Average Directional Accuracy: 51.62%
 Average Profitable Trades: 53.00%
 Average Final Capital: $10007.63 (± $10.43)
 Average #Trades: 14.00 | Avg Trade Return: 0.0070%

 ===== Strategy: top_quartile — 1000 runs =====
 Average Total Return: 0.11% (± 0.10%, p05 -0.07%, median 0.11%, p95 0.27%)
 Average Directional Accuracy: 51.60%
 Average Profitable Trades: 52.28%
 Average Final Capital: $10010.81 (± $10.36)
 Average #Trades: 8.00 | Avg Trade Return: 0.0149%
🧮 Model: GPT-3.5 Regime-Sensitiv

100%|███████████████████████████████████████████████████████████████████████████| 94/94 [04:25<00:00,  2.83s/it]



 ===== LLM Statistical Metrics (noiseless) =====
RMSE: 775494191.680574
MAE: 144006358.146464
R²: -1812931481694162846220288.0000

 ===== Strategy: threshold — 1000 runs =====
 Average Total Return: 0.33% (± 0.08%, p05 0.20%, median 0.33%, p95 0.46%)
 Average Directional Accuracy: 51.17%
 Average Profitable Trades: 52.76%
 Average Final Capital: $10033.44 (± $8.19)
 Average #Trades: 12.24 | Avg Trade Return: 0.0320%

 ===== Strategy: relative — 1000 runs =====
 Average Total Return: 0.17% (± 0.10%, p05 -0.00%, median 0.18%, p95 0.33%)
 Average Directional Accuracy: 51.16%
 Average Profitable Trades: 52.12%
 Average Final Capital: $10017.34 (± $10.36)
 Average #Trades: 14.00 | Avg Trade Return: 0.0153%

 ===== Strategy: top_quartile — 1000 runs =====
 Average Total Return: 0.18% (± 0.08%, p05 0.06%, median 0.18%, p95 0.30%)
 Average Directional Accuracy: 51.13%
 Average Profitable Trades: 51.84%
 Average Final Capital: $10018.09 (± $7.60)
 Average #Trades: 8.00 | Avg Trade Return: 0.02

In [27]:
# ======================================
# Test GPT-4 Model
# ======================================

print("\n TESTING GPT-4 MODEL")
print("=" * 50)
gpt4_models = {'LLMTime GPT-4': gpt4_hypers, 
     'GPT-4 Volatility-Aware': gpt4_volatility_aware,
     'GPT-4 Conservative':gpt4_conservative}
all_gpt4_results = dict()

for model_name, params in gpt4_models.items():
    print(f"🧮 Model: {model_name}")
    all_gpt4_results[model_name] = predict_returns_with_trading(
        splits, 
        model_type="llm", 
        forecast_horizon=FORECAST_HORIZON, 
        hypers=gpt4_hypers,
        initial_capital=INITIAL_CAPITAL,
        verbose=False
    )
    print("=" * 40)


 TESTING GPT-4 MODEL
🧮 Model: LLMTime GPT-4
Starting hyperparameter tuning over 1 configurations...
Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gpt-4', 'alpha': 0.3, 'basic': True, 'temp': 1.0, 'top_p': 0.8, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=', ', bit_sep='', plus_sign='', minus_sign='-', half_bin_correction=True, decimal_point='', missing_str=' Nan')}) 
 with NLL inf













  0%|                                                                                    | 0/94 [00:00<?, ?it/s]

KeyboardInterrupt: 

#### Discussion of Results

For each model, the best-performing configuration is selected, with **Directional Accuracy used as the primary criterion** for evaluation and comparison. However, all models—regardless of configuration—produced similar overall results, highlighting the **limited variation in performance across different setups**.

In [ ]:
arima_results = all_arima_results["ARIMA"]
gpt3_results = all_gpt3_results["LLMTime GPT-3.5"]
gpt4_results = all_gpt4_results["GPT-4 Volatility-Aware"]
comparison_results = {
    'arima': arima_results,
    'gpt3': gpt3_results, 
    'gpt4': gpt4_results
}

In [ ]:
summary_df = create_performance_summary_table(comparison_results)

In this comparison, the **statistical metrics** (RMSE, MAE, and R²) are not the main focus, since the goal is **not to predict the exact return values**. They are added to complete the full analysis, however what matters most are the **Trading Protocol metrics**, especially the **Directional Accuracy**.

##### 🔍 Observations

Overall, the three models behave quite similarly, and **none delivers a strong predictive capability**. They all show **directional accuracy close to random** (around **55–62%**), meaning none of them consistently predicts the right direction much better than chance.  

These results prove the statement the **difficulty of financial forecasting**, where even advanced models struggle to outperform random direction guessing.  

#### Visualization of Results

##### 📊 Plot Predictions vs Actual Returns

In [ ]:
print(len(arima_results['actual']))
print(len(arima_results['actual'][0]))
print(len(gpt3_results['actual']))
print(len(gpt3_results['actual'][0]))

In [ ]:
plot_predictions_vs_actual(comparison_results, forecast_horizon=FORECAST_HORIZON)

In this project, it is **expected** that the models will not precisely match the actual return series values. The purpose of this plot is mainly to visualize the predicted versus actual returns and not to assess value accuracy. 

We can observe a **linear and smoothed trend** from the **ARIMA model**, which is consistent with its statistical and stationary assumptions. In contrast, **GPT-3.5** and **GPT-4** display a **more dynamic and nonlinear behavior**, producing sharper fluctuations and less stable trends.

Despite the differences in predicted values, all models show a **similar level of directional understanding**, which is close to random performance. This emphasizes the inherent difficulty of forecasting stock returns providing only the historic time-series data.

#### 💰 Cumulative Trading Performance Comparison

In [ ]:
plot_cumulative_returns_simulation(comparison_results, initial_capital=INITIAL_CAPITAL)

This plot shows the **cumulative portfolio value** over the 30-day trading horizon for different trading approaches:  
**Buy & Hold**, **ARIMA-based trading**, **GPT-3.5 trading**, and **GPT-4 trading**.

_Definition_:
The **Buy & Hold** strategy (gray dashed line) represents a **baseline investor** who invests once at the beginning and holds the asset throughout the entire period without making any trades.  


The LLM-based models (GPT-3.5 and GPT-4) show more active trading behavior, but their results are unstable and inconsistent, reflecting the randomness and noise in real market movements. The Buy & Hold strategy still performs better than all model-based approaches, which is common in stock forecasting.

In the end, even with advanced models, predicting market returns doesn’t always lead to better trading results, highlighting just how unpredictable and challenging stock markets really are.

#### Summary Table

In [ ]:
display(create_performance_summary_table(comparison_results))

## Conclusion

This project explored the application of Large Language Models (LLMs) for stock market prediction, specifically focusing on S&P 500 returns forecasting. The research compared traditional statistical methods (ARIMA) with modern LLM-based approaches (GPT-3.5 and GPT-4) using a evaluation framework that included statistical metrics, but focuses more on practical trading performance assessment.

### 🎯 **Key Findings**

1. **Limited Predictive Performance Across All Models**
   - All models demonstrated **directional accuracy close to random** (55-62%), indicating that none consistently outperformed chance in predicting market direction
   - Statistical metrics (RMSE, MAE, R²) showed poor fit across all approaches, confirming the challenging nature of financial time series prediction

2. **Model Behavior Differences**
   - **ARIMA** produced **linear and smoothed predictions**, consistent with its statistical assumptions and stationarity requirements
   - **LLM models** (GPT-3.5 and GPT-4) exhibited **more dynamic and nonlinear behavior** with sharper fluctuations, potentially capturing complex patterns but failing to translate into superior accuracy
   - Despite different prediction patterns, all models converged to similar **overall performance levels**

### 🔍 **Implications for Financial Forecasting**

The results reinforce the **Efficient Market Hypothesis** and highlight several critical insights:

- **Market Complexity**: Even sophisticated models like GPT-4 struggle with the inherent randomness and complexity of financial markets
- **Data Limitations**: Historical return patterns may not contain sufficient predictive signals for future performance
- **Model Limitations**: Both statistical and deep learning approaches face challenges when applied to highly volatile, non-stationary financial data
